In [1]:
import cudf
import cuspatial as csp

In [2]:
%%time
df = cudf.read_parquet("../../US_MARINE/ALL_GOM_tanker_2019-2021.parquet")[['BaseDateTime', 'MMSI', 'LAT', 'LON']].reset_index(drop=True)
df['MMSI'] = df.MMSI.astype('int32')
df['BaseDateTime'] = df.BaseDateTime.astype('datetime64[ms]')
df = df.sort_values(by=['MMSI','BaseDateTime']).reset_index(drop=True)
df
# objects, traj_offsets = csp.derive_trajectories(df.MMSI, df.LON, df.LAT, df.Time)
# objects

CPU times: user 677 ms, sys: 389 ms, total: 1.07 s
Wall time: 2.4 s


,BaseDateTime,MMSI,LAT,LON
0,2019-01-02 19:05:00,3791472,26.13211,-92.04048
1,2019-01-02 19:10:00,3791472,26.13213,-92.04049
2,2019-01-02 19:15:00,3791472,26.13215,-92.04044
3,2019-01-02 19:20:00,3791472,26.13216,-92.04049
4,2019-01-02 19:25:00,3791472,26.13212,-92.04046
...,...,...,...,...
38903728,2021-06-22 13:50:00,710033290,28.19147,-94.51869
38903729,2021-06-22 14:50:00,710033290,28.00145,-94.46387
38903730,2021-06-22 15:30:00,710033290,27.87318,-94.41796
38903731,2020-01-24 15:25:00,866666666,29.42988,-93.64103


In [3]:
mmsi_names = df.MMSI.unique().to_arrow().to_pylist()
mmsi_names[:5]

[3791472, 205042000, 205086000, 205089000, 205097000]

In [11]:
import time
import datetime
def minus_2019_ts(ts):
    try:
        ans = ts.astype('datetime64[s]').astype('int') - time.mktime(datetime.datetime(2019,1,1).timetuple())
    except:
        ans = -9999.0
    return ans
minus_2019_ts(df.iloc[0].BaseDateTime)

0    140700.0
Name: BaseDateTime, dtype: float64

In [15]:
df['Unix_Seconds'] = df.BaseDateTime.astype('datetime64[s]').astype('int') - 1546315200.0 # 2019-1-1 in seconds
df

,BaseDateTime,MMSI,LAT,LON,Unix_Seconds
0,2019-01-02 19:05:00,3791472,26.13211,-92.04048,140700.0
1,2019-01-02 19:10:00,3791472,26.13213,-92.04049,141000.0
2,2019-01-02 19:15:00,3791472,26.13215,-92.04044,141300.0
3,2019-01-02 19:20:00,3791472,26.13216,-92.04049,141600.0
4,2019-01-02 19:25:00,3791472,26.13212,-92.04046,141900.0
...,...,...,...,...,...
38903728,2021-06-22 13:50:00,710033290,28.19147,-94.51869,78054600.0
38903729,2021-06-22 14:50:00,710033290,28.00145,-94.46387,78058200.0
38903730,2021-06-22 15:30:00,710033290,27.87318,-94.41796,78060600.0
38903731,2020-01-24 15:25:00,866666666,29.42988,-93.64103,33564300.0


In [32]:
%%time
list_of_rows = []
for j, mmsi in enumerate(mmsi_names[:]):
    tmp = df[df.MMSI == mmsi]
    list_of_paths = []
    list_of_ts = []
    for i,row in tmp.to_pandas().iterrows():
        list_of_paths.append([row.LON,row.LAT])
        list_of_ts.append(row.Unix_Seconds)
    list_of_rows.append([mmsi, list_of_paths, list_of_ts])
    if j % 20 == 0:
        print(j, mmsi)

0 3791472
20 205760000
40 215077000
60 215503000
80 215669000
100 215894000
120 219027876
140 219156000
160 219322000
180 219603000
200 220629000
220 226099000
240 229248000
260 229582000
280 232006300
300 235009410
320 235076274
340 235108529
360 236702000
380 240235000
400 240728000
420 240991000
440 241330000
460 241529000
480 241632000
500 241734000
520 247293200
540 248372000
560 248754000
580 249084000
600 249592000
620 249882000
640 255806138
660 256338000
680 257034490
700 257326000
720 257447000
740 257872000
760 258749000
780 259773000
800 271043022
820 308871000
840 309992000
860 310652000
880 310800000
900 311000333
920 311000476
940 311001007
960 311026700
980 311498000
1000 316011550
1020 319280000
1040 319627000
1060 351896000
1080 352901000
1100 353616000
1120 354628000
1140 355364000
1160 356186000
1180 357315000
1200 366495000
1220 368589000
1240 370642000
1260 371294000
1280 371809000
1300 372861000
1320 373592000
1340 374788000
1360 403548000
1380 419001224
1400 477

In [33]:
traj_df = cudf.DataFrame(list_of_rows, columns=['mmsi','path','ts'])
traj_df.to_parquet("trajectories/ALL_GOM_tanker_2019-2021_traj.parquet")

In [34]:
traj_df

,mmsi,path,ts
0,3791472,"[[-92.04048, 26.13211], [-92.04049, 26.13213],...","[140700.0, 141000.0, 141300.0, 141600.0, 14190..."
1,205042000,"[[-89.2095, 27.24959], [-89.26068, 27.33388], ...","[80076600.0, 80077800.0, 80078100.0, 80078400...."
2,205086000,"[[-93.48448, 26.39778], [-93.56166, 26.50175],...","[77814000.0, 77816700.0, 77819400.0, 77821800...."
3,205089000,"[[-90.35513, 24.80777], [-90.36815, 24.81126],...","[94534200.0, 94535100.0, 94536000.0, 94536300...."
4,205097000,"[[-93.45964, 26.5793], [-93.6274, 26.76886], [...","[82272300.0, 82275300.0, 82275600.0, 82276200...."
...,...,...,...
2972,642122021,"[[-93.82248, 25.65528], [-93.85108, 25.66127],...","[19596300.0, 19596900.0, 19597500.0, 19597800...."
2973,645337000,"[[-91.57331, 26.81737], [-91.60873, 26.85394],...","[17980500.0, 17981400.0, 17982600.0, 17982900...."
2974,710016250,"[[-88.35071, 25.61065], [-88.36058, 25.62274],...","[12231000.0, 12231300.0, 12231600.0, 12231900...."
2975,710033290,"[[-92.98647, 26.33258], [-93.03388, 26.37598],...","[76974900.0, 76976100.0, 76976400.0, 76977000...."
